In [1]:
import pandas as pd
import numpy as np

from pandasql import sqldf

This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is <a href=https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh>here</a>, and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [2]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.4260,39.0,165.0,3025654.0,3.010850e+09
5,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,9.884187e+05,219825.5227,3.0,145.0,1076229.0,1.011310e+09
6,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,9.883112e+05,219885.2785,3.0,145.0,1076229.0,1.011310e+09
7,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,9.877691e+05,217157.8561,3.0,133.0,1086093.0,1.010410e+09
8,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,9.630732e+05,156635.5542,NaN,NaN,NaN,NaN
9,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,9.926537e+05,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


Looks good, but lets take a look at the shape of this data:

In [5]:
trees.shape

(1000, 45)

1000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset.

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [4]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29,739,4052307,4022210001
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19,973,4101931,4044750045
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34,449,3338310,3028870001
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34,449,3338342,3029250001
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39,165,3025654,3010850052


If we wanted the next 5, we would use this url:

In [7]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,190422,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770046,-73.984950,988418.6997,219825.5227,3.0,145.0,1076229.0,1.011310e+09
1,190426,106099,2015-08-30,11,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,1014500,New York,40.770210,-73.985338,988311.1900,219885.2785,3.0,145.0,1076229.0,1.011310e+09
2,208649,103940,2015-09-07,9,0,OnCurb,Alive,Good,Tilia americana,American linden,...,1012700,New York,40.762724,-73.987297,987769.1163,217157.8561,3.0,133.0,1086093.0,1.010410e+09
3,209610,407443,2015-09-08,6,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,5006400,New York,40.596579,-74.076255,963073.1998,156635.5542,NaN,NaN,NaN,NaN
4,192755,207508,2015-08-31,21,0,OffsetFromCurb,Alive,Fair,Platanus x acerifolia,London planetree,...,3037402,New York,40.586357,-73.969744,992653.7253,152903.6306,47.0,37402.0,3320727.0,3.072350e+09


You can read more about paging using the Socrata API <a href=https://dev.socrata.com/docs/paging.html>here</a>

In these docs, you'll also see more advanced functions (called `SoQL`) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using `SoQL` is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

In [3]:
boro = 'Bronx'
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

,count_tree_id,spc_common
0,4619,NaN
1,43,black walnut
2,9,spruce
3,62,tulip-poplar
4,7,trident maple
...,...,...
128,195,crimson king maple
129,774,purple-leaf plum
130,363,sawtooth oak
131,505,willow oak


This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

-----

**NOTE**: One tip in dealing with URLs: you may need to replace spaces with `'%20'`. I personally just write out the url and then follow the string with a replace:

In [9]:
'https://api-url.com/?query with spaces'.replace(' ', '%20')

'https://api-url.com/?query%20with%20spaces'

## Assignment 4

In this module we’ll be looking at data from the New York City tree census: 
https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh 

This data is collected by volunteers across the city, and is meant to catalog information about every single tree in the city. 

Build a dash app for a arborist studying the health of various tree species (as defined by the variable ‘spc_common’) across each borough (defined by the variable ‘borough’). This arborist would like to answer the following two questions for each species and in each borough:

1. What proportion of trees are in good, fair, or poor health according to the ‘health’ variable?
2. Are stewards (steward activity measured by the ‘steward’ variable) having an impact on the health of trees? 

Please see the accompanying notebook for an introduction and some notes on the Socrata API. 

**Deployment**: Dash deployment is more complicated than deploying shiny apps, so deployment in this case is optional (and will result in extra credit). You can read instructions on deploying a dash app to heroku here: https://dash.plot.ly/deployment

In [4]:
url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=boroname,health, count(tree_id)' +\
        '&$group=boroname,health').replace(' ', '%20')

q1 = pd.read_json(url)

q1

,boroname,health,count_tree_id
0,Bronx,Fair,10887
1,Bronx,Good,66603
2,Bronx,Poor,3095
3,Bronx,NaN,4618
4,Brooklyn,Fair,25073
5,Brooklyn,Good,138212
6,Brooklyn,Poor,6459
7,Brooklyn,NaN,7549
8,Manhattan,Fair,11460
9,Manhattan,Good,47358


Get the count of trees per condition

In [7]:
#Count of trees per boro,spc, health and status

soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=count(tree_id),boroname,spc_common, health,status' +\
         '&$where=health!=\'NaN\'' +\
        '&$group=boroname,health,status,spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

In [8]:
#Count of trees per boro, health and steward
soql_url_2 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=count(tree_id), boroname, health, steward' +\
         '&$where=health!=\'NaN\'' +\
        '&$group=health,steward, boroname').replace(' ', '%20')
soql_trees_2 = pd.read_json(soql_url_2)

In [9]:
#Count of trees per spc , health and steward 
soql_url_3 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=count(tree_id), spc_common, health, steward' +\
         '&$where=health!=\'NaN\'' +\
        '&$group=health,steward, spc_common').replace(' ', '%20')
soql_trees_3 = pd.read_json(soql_url_3)

In [10]:
soql_trees_sum_2 = soql_trees_2.groupby(['boroname', 'health']).agg({'count_tree_id': [np.sum]})

soql_trees_sum_2b=pd.DataFrame(soql_trees_sum_2.to_records())

In [11]:
soql_merged_2 = pd.merge(soql_trees_2, soql_trees_sum_2b, on=['boroname', 'health'])


soql_merged_2.columns = ['count_tree_id', 'boroname', 'health','steward', 'sum_count_tree_id']

soql_merged_2.head()

,count_tree_id,boroname,health,steward,sum_count_tree_id
0,2130,Bronx,Fair,1or2,10887
1,125,Bronx,Fair,3or4,10887
2,7,Bronx,Fair,4orMore,10887
3,8625,Bronx,Fair,None,10887
4,6490,Brooklyn,Fair,1or2,25073


In [12]:
soql_trees_sum_3 = soql_trees_3.groupby(['spc_common', 'health']).agg({'count_tree_id': [np.sum]})

soql_trees_sum_3b=pd.DataFrame(soql_trees_sum_3.to_records())

soql_merged_3 = pd.merge(soql_trees_3, soql_trees_sum_3b, on=['spc_common','health'])

soql_merged_3.columns = ['count_tree_id',  'spc_common','health','steward', 'sum_count_tree_id']

soql_merged_3.head()

,count_tree_id,spc_common,health,steward,sum_count_tree_id
0,19,American beech,Fair,1or2,38
1,1,American beech,Fair,3or4,38
2,18,American beech,Fair,None,38
3,335,American elm,Fair,1or2,1295
4,34,American elm,Fair,3or4,1295


In [13]:
soql_trees_sum = soql_trees.groupby(['boroname', 'spc_common']).agg({'count_tree_id': [np.sum]})

soql_trees_sumb=pd.DataFrame(soql_trees_sum.to_records())

soql_merged = pd.merge(soql_trees, soql_trees_sumb, on=['boroname','spc_common'])

soql_merged.columns = ['count_tree_id','boroname', 'spc_common', 'health', 'status', 'sum_count_tree_id']

soql_merged.head()

,count_tree_id,boroname,spc_common,health,status,sum_count_tree_id
0,7,Bronx,American beech,Fair,Alive,31
1,21,Bronx,American beech,Good,Alive,31
2,3,Bronx,American beech,Poor,Alive,31
3,248,Bronx,American elm,Fair,Alive,1471
4,1176,Bronx,American elm,Good,Alive,1471


In [14]:
soql_merged['prop_health'] = soql_merged['count_tree_id'] / soql_merged['sum_count_tree_id']

soql_merged.sort_values(by=['boroname','spc_common'])

q1 = soql_merged[['boroname','health','spc_common','prop_health']]

q1.head(10)

,boroname,health,spc_common,prop_health
0,Bronx,Fair,American beech,0.225806
1,Bronx,Good,American beech,0.677419
2,Bronx,Poor,American beech,0.096774
3,Bronx,Fair,American elm,0.168593
4,Bronx,Good,American elm,0.799456
5,Bronx,Poor,American elm,0.031951
6,Bronx,Fair,American hophornbeam,0.151351
7,Bronx,Good,American hophornbeam,0.800000
8,Bronx,Poor,American hophornbeam,0.048649
9,Bronx,Fair,American hornbeam,0.206780


In [15]:
soql_merged_2['prop_steward'] = soql_merged_2['count_tree_id'] / soql_merged_2['sum_count_tree_id']

soql_merged_2.sort_values(by=['boroname','steward'])

q2 = soql_merged_2

q2.head(10)

,count_tree_id,boroname,health,steward,sum_count_tree_id,prop_steward
0,2130,Bronx,Fair,1or2,10887,0.195646
1,125,Bronx,Fair,3or4,10887,0.011482
2,7,Bronx,Fair,4orMore,10887,0.000643
3,8625,Bronx,Fair,None,10887,0.792229
4,6490,Brooklyn,Fair,1or2,25073,0.258844
5,760,Brooklyn,Fair,3or4,25073,0.030311
6,59,Brooklyn,Fair,4orMore,25073,0.002353
7,17764,Brooklyn,Fair,None,25073,0.708491
8,4471,Manhattan,Fair,1or2,11460,0.390140
9,1415,Manhattan,Fair,3or4,11460,0.123473


In [16]:
soql_merged_3['prop_steward'] = soql_merged_3['count_tree_id'] / soql_merged_3['sum_count_tree_id']

soql_merged_3.sort_values(by=['spc_common','steward'])

q3 = soql_merged_3

q3.head(10)

,count_tree_id,spc_common,health,steward,sum_count_tree_id,prop_steward
0,19,American beech,Fair,1or2,38,0.500000
1,1,American beech,Fair,3or4,38,0.026316
2,18,American beech,Fair,None,38,0.473684
3,335,American elm,Fair,1or2,1295,0.258687
4,34,American elm,Fair,3or4,1295,0.026255
5,4,American elm,Fair,4orMore,1295,0.003089
6,922,American elm,Fair,None,1295,0.711969
7,48,American hophornbeam,Fair,1or2,158,0.303797
8,5,American hophornbeam,Fair,3or4,158,0.031646
9,1,American hophornbeam,Fair,4orMore,158,0.006329


## Proportion of health by Borough per SPC

In [21]:
#  https://dash.plotly.com/deployment


import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

df = q1

available_indicators = df['spc_common'].unique()

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([
    html.H1('Proportion of health by Borough per SPC'),
    html.Div('''
        SPC
    '''),
    dcc.Dropdown(
        id='my-dropdown',
        options=[{'label': i, 'value': i} for i in available_indicators],
       
    ),
    dcc.Graph(
        id='example-graph'    
    )
    
])

@app.callback(
    dash.dependencies.Output('example-graph', 'figure'),
    [dash.dependencies.Input('my-dropdown', 'value')])

def update_output(selected_dropdown_value):
    dff = df[df['spc_common'] == selected_dropdown_value]
    figure = {
            'data': [
                {'x': dff.boroname[dff['health'] == 'Good'], 'y': dff.prop_health[dff['health'] == 'Good'], 'type': 'bar', 'name': 'Good'},
                {'x': dff.boroname[dff['health'] == 'Fair'], 'y': dff.prop_health[dff['health'] == 'Fair'], 'type': 'bar', 'name': 'Fair'},
                {'x': dff.boroname[dff['health'] == 'Poor'], 'y': dff.prop_health[dff['health'] == 'Poor'], 'type': 'bar', 'name': 'Poor'}
            ],
      
        }
    return figure 


if __name__ == '__main__':
    app.run_server()

127.0.0.1 - - [27/Oct/2022 09:32:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:32:23] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_2m1666774906.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:32:23] "GET /_dash-component-suites/dash/deps/react@16.v2_6_2m1666774906.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:32:23] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_2m1666774906.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:32:23] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_2m1666774906.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:32:23] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_2m1666774906.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:32:23] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_6_2m1666774906.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:32:23] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_6_2m1666774906.js HTT

### I am unable to host the app on heroku, attached below is the screenshot from my local hosting.

![1](1.jpg)

## Proportion of Steward by health per Borough

In [25]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

df = q2

available_indicators = df['boroname'].unique()

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([
    html.H1('Proportion of Steward by health per Borough'),
    html.Div('''
        Borough
    '''),
    dcc.Dropdown(
        id='my-dropdown',
        options=[{'label': i, 'value': i} for i in available_indicators],
        
    ),
    dcc.Graph(
        id='example-graph'    
    )
    
])

@app.callback(
    dash.dependencies.Output('example-graph', 'figure'),
    [dash.dependencies.Input('my-dropdown', 'value')])

def update_output(selected_dropdown_value):
    dff = df[df['boroname'] == selected_dropdown_value]
    figure = {
            'data': [
                {'x': dff.health[dff['steward'] == 'None'], 'y': dff.prop_steward[dff['steward'] == 'None'], 'type': 'bar', 'name': 'None'},
                {'x': dff.health[dff['steward'] == '1or2'], 'y': dff.prop_steward[dff['steward'] == '1or2'], 'type': 'bar', 'name': '1or2'},
                {'x': dff.health[dff['steward'] == '3or4'], 'y': dff.prop_steward[dff['steward'] == '3or4'], 'type': 'bar', 'name': '3or4'},
                {'x': dff.health[dff['steward'] == '4orMore'], 'y': dff.prop_steward[dff['steward'] == '4orMore'], 'type': 'bar', 'name': '4orMore'}
            ],
            
        }
    return figure 


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Oct/2022 09:34:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:34:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:34:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:34:18] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:34:18] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:34:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:34:18] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:34:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:34:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:34:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:35:00] "POST /_dash-update-component HTTP/1.

![2](2.jpg)

## Proportion of Steward by health per SPC

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

df = q3

available_indicators = df['spc_common'].unique()

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([
    html.H1('Proportion of Steward by health per SPC'),
    html.Div('''
        SPC
    '''),
    dcc.Dropdown(
        id='my-dropdown',
        options=[{'label': i, 'value': i} for i in available_indicators],
       
    ),
    dcc.Graph(
        id='example-graph'    
    )
    
])

@app.callback(
    dash.dependencies.Output('example-graph', 'figure'),
    [dash.dependencies.Input('my-dropdown', 'value')])

def update_output(selected_dropdown_value):
    dff = df[df['spc_common'] == selected_dropdown_value]
    figure = {
            'data': [
                {'x': dff.health[dff['steward'] == 'None'], 'y': dff.prop_steward[dff['steward'] == 'None'], 'type': 'bar', 'name': 'None'},
                {'x': dff.health[dff['steward'] == '1or2'], 'y': dff.prop_steward[dff['steward'] == '1or2'], 'type': 'bar', 'name': '1or2'},
                {'x': dff.health[dff['steward'] == '3or4'], 'y': dff.prop_steward[dff['steward'] == '3or4'], 'type': 'bar', 'name': '3or4'},
                {'x': dff.health[dff['steward'] == '4orMore'], 'y': dff.prop_steward[dff['steward'] == '4orMore'], 'type': 'bar', 'name': '4orMore'}
            ],
            
        }
    return figure 


if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Oct/2022 09:39:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:39:32] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_2m1666774906.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:39:32] "GET /_dash-component-suites/dash/deps/react@16.v2_6_2m1666774906.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:39:32] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_2m1666774906.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:39:32] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_2m1666774906.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:39:32] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_2m1666774906.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:39:32] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_6_2m1666774906.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Oct/2022 09:39:32] "GET /_dash-component-suites/d

![3](3.jpg)